In [16]:
import numpy as np
import sys
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import load_model
import tensorflow.keras.losses
from keras.utils import to_categorical

from constants import * 
from heuristic import *
from io_help import *
from solver import *

def one_hot_encode(board):
    """ 
    This function one hot encodes the board into a length 256 array.
    The one hot encoding gives the location of each number in the board.
    For example, the first 16 of the 256 numbers will indicate where on
    the board the 1 tile is. 
    """

    flat = (board.reshape(SIZE ** 2)).tolist()

    X = []
    for i in np.arange(1,17): 
        encoding = np.zeros(SIZE ** 2)
        encoding[flat.index(i)] = 1

        X.append(encoding)  

    X = (np.asarray(X).reshape(SIZE ** 4))

    # Potentially append Manhattan distance. 
    # np.append(X, manhattan(board))

    return X

def load_data(file_name):
    """
    This function reads in training data from a file and returns 
    the one-hot encoded data X and their labels Y as a tuple. 
    """
    file = open(file_name, "r")

    X = []
    Y = []

    for string in file: 
        (board, classification) = string_to_board_and_dist(string) 

        X.append(one_hot_encode(board))
        Y.append(classification)

    file.close()

    return(np.asarray(X),np.asarray(Y))

def evaluate(X,Y):
    """
    This function reads in training data from a file and 
    trains and evaluates NN model using kfold validation. 
    """
    #(X,Y) = load_data(file_name)

    #Y = to_categorical(Y)

    # Implement K-fold cross validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=2020)

    for train, test in kfold.split(X, Y):
        # Build Model
        model = Sequential()

        # Input Layer
        i = Input(shape = (256,))
        x_1 = Dense(256, activation='relu')(i)
        #x_2 = Dropout(0.2)(x_1)
        o = Dense(5, activation='softmax')(x_1)
        model = Model(i,o)

        # Define the optimizer and loss function
        model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

        # Train 
        model.fit(X, Y, epochs=15, verbose=1)

        # Evaluate
        score = model.evaluate(X[test], Y[test], verbose=0)
        print(score)

    return model

def train(X,Y):
    """
    This function reads in training data from a file and returns a 
    trained NN model. 
    """
    #(X,Y) = load_data(file_name)

    #Y = to_categorical(Y)

    # Build Model
    model = Sequential()

    # Input Layer
    i = Input(shape = (256,))
    x_1 = Dense(256, activation='relu')(i)
    #x_2 = Dropout(0.2)(x_1)
    o = Dense(5, activation='softmax')(x_1)
    model = Model(i,o)

    # Define the optimizer and loss function
    model.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['accuracy'])

    # Train 
    model.fit(X, Y, epochs=50, verbose=1)

    return model

In [17]:
(X,Y) = load_data("final_portfolio_data.txt")

In [18]:
model = train(X,Y)

board = gen_board()
print(board)
print(model.predict(one_hot_encode(board).reshape(1,256)))

Train on 395715 samples
Epoch 1/50
395715/395715 [==============================] - 19s 47us/sample - loss: 0.6937 - accuracy: 0.6926 - loss: 0.6938 - accuracy: 0.69
Epoch 2/50
395715/395715 [==============================] - 19s 47us/sample - loss: 0.6419 - accuracy: 0.7138 - loss: 0.6420 - 
Epoch 3/50
395715/395715 [==============================] - 19s 47us/sample - loss: 0.6176 - accuracy: 0.7263
Epoch 4/50
395715/395715 [==============================] - 19s 48us/sample - loss: 0.5998 - accuracy: 0.7365
Epoch 5/50
395715/395715 [==============================] - 21s 53us/sample - loss: 0.5856 - accuracy: 0.7439
Epoch 6/50
395715/395715 [==============================] - 20s 50us/sample - loss: 0.5747 - accuracy: 0.7500
Epoch 7/50
395715/395715 [==============================] - 23s 58us/sample - loss: 0.5653 - accuracy: 0.7546
Epoch 8/50
395715/395715 [==============================] - 17s 43us/sample - loss: 0.5576 - accuracy: 0.7586 - loss: 0
Epoch 9/50
395715/395715 [==========

395715/395715 [==============================] - 17s 44us/sample - loss: 0.4830 - accuracy: 0.7978
Epoch 44/50
395715/395715 [==============================] - 17s 44us/sample - loss: 0.4824 - accuracy: 0.7983
Epoch 45/50
395715/395715 [==============================] - 18s 45us/sample - loss: 0.4817 - accuracy: 0.7985
Epoch 46/50
395715/395715 [==============================] - 19s 47us/sample - loss: 0.4810 - accuracy: 0.7993 - loss: 0.4805 - accuracy: 0.79 - ETA: 0s -
Epoch 47/50
395715/395715 [==============================] - 20s 50us/sample - loss: 0.4804 - accuracy: 0.7997
Epoch 48/50
395715/395715 [==============================] - 18s 46us/sample - loss: 0.4798 - accuracy: 0.8001
Epoch 49/50
395715/395715 [==============================] - 17s 44us/sample - loss: 0.4794 - accuracy: 0.8000
Epoch 50/50
395715/395715 [==============================] - 17s 44us/sample - loss: 0.4787 - accuracy: 0.8010 - loss: 0.4786 - accuracy: 
[[ 2  3  7  4]
 [ 9  5  8 16]
 [ 6  1 14 11]
 [13 15

In [19]:
model.save("algorithm_model_50_no_do.h5")

In [20]:
model2 = load_model("algorithm_model_50_no_do.h5")
model2.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 256)]             0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_19 (Dense)             (None, 5)                 1285      
Total params: 67,077
Trainable params: 67,077
Non-trainable params: 0
_________________________________________________________________


In [29]:
board = gen_board()
print(board)
print(np.asarray(model.predict(one_hot_encode(board).reshape(1,256)).flatten()))
print(np.asarray(model.predict(one_hot_encode(board).reshape(1,256)).flatten()).argmax())



[[ 1  3  6  4]
 [ 9 16  5  7]
 [15 14 10  8]
 [11 13  2 12]]
[6.2154625e-02 1.5069728e-01 7.8709257e-01 3.2622894e-19 5.5535209e-05]
2
